# CADC Astroquery
Astroquery is package that provides a set of tools to query astronomical databases. In this tutorial, we will focus on the CADC package of astroquery, which queries data provided by the Canadian Astronomical Data Centre (http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca ). This tutorial will go through an example of runing a query to displaying an image. We will gather images from HST and CFHT, do a cutout of the area we want, and display the the images. (Note: I haven't added PansSTARRS DR1g data yet)


# Tutorial
This tutorial will go through some of the basic functionalities of the CADC astroquery package. The CADC module can be installed using the command:

```
$ pip install --pre astroquery
```

(The `--pre` tag installs the pre-release version which we need because the CADC package is not in the latest release.) Alternatively, you can clone and install from the source:
TODO: Remove $

```
$ # If you have a github account:
$ git clone git@github.com:astropy/astroquery.git
$ # If you do not:
$ git clone https://github.com/astropy/astroquery.git
$ cd astroquery
$ python setup.py install
```

More information about astroquery can be found at the github repository https://github.com/astropy/astroquery.

Now, onto the tutorial. The first step is to import the CADC module from astroquery.

In [1]:
from astroquery.cadc import Cadc
cadc = Cadc()


## Querying

TODO: Ask what energy band for HST images


Now we can query the CADC database and retreive the data we want. Let's take a look at MegaPipe data in the D2 field, and compare it to HST images in the same area. The MegaPipe collection is data collected by the CFHT MegaCam and stacked at the CADC (http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/cfht/). The D2 field is in the region `10:00:28 02:12:30` (https://www.cfht.hawaii.edu/Science/CFHLS/cfhtlsdeepwidefields.html) so let's query around that area for both CFHT and HST data. We also want to restrict results to calibrated images of good quality.

For MegaPipe images, we want energy bands in the R ($\lambda_{eff} = 658 nm$) or I ($\lambda_{eff} = 806 nm$) range (https://en.wikipedia.org/wiki/Photometric_system).

For HST images, energy bands use the WFC system, and right now I'm not sure which filter we would like (http://www.stsci.edu/hst/wfc3/ins_performance/ground/components/filters).

First we grab the coordinates of the D2 field:


In [11]:
from astropy.coordinates import SkyCoord

c = SkyCoord('10h00m28s', '02d12m30s', frame='icrs', unit='deg')
radius = 0.0166
# QUERY Same supernova, different bands
# display all bands side by side
# 3000 - 10000 ang ['energy_bounds_lower', 'energy_bounds_upper']
print(c)

<SkyCoord (ICRS): (ra, dec) in deg
    (150.11666667, 2.20833333)>


### Method 1: Query Region then Filter
The simplest way to query data by region is to use the `query_region` function. It takes coordinates and a collection source (optional) and returns all the results that contain the cordinates. If radius is passed in, then it returns results that intersect with that area. After querying, we can further filter down the results using boolean masks on the data.


In [55]:
results = cadc.query_name('SNLS 05D2ah')
#

In [61]:
query = '''SELECT *
FROM caom2.Plane AS Plane 
JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID 
WHERE ( Observation.target_name LIKE '%05D2ah%' )'''

job = cadc.run_query(query, 'sync')
results = job.get_results()

In [67]:
results['energy_bounds_lower', 'energy_bounds_upper']

energy_bounds_lower,energy_bounds_upper
m,m
float64,float64
4.940000000000001e-07,6.300000000000001e-07
4.940000000000001e-07,6.300000000000001e-07
4.940000000000001e-07,6.300000000000001e-07
4.940000000000001e-07,6.300000000000001e-07
4.716100000000001e-07,8.885200000000001e-07
4.716100000000001e-07,8.885200000000001e-07


In [15]:
cfht_results = cadc.query_region(c, radius=radius, collection='CFHTMEGAPIPE')
hst_results = cadc.query_region(c, radius=radius, collection='HST')

print('Number of CFHT results: {}'.format(len(cfht_results)))
print('Number of HST results: {}'.format(len(hst_results)))

Number of CFHT results: 142
Number of HST results: 537


Let's take a look at the column names of the results to see what we can filter down. In this case, we want to look at the `calibrationLevel`, `dataProductType`, `quality_flag`, `target_name`, and `energy_bandpassName` columns.

In [13]:
cfht_results.colnames

['caomObservationURI',
 'sequenceNumber',
 'proposal_keywords',
 'target_standard',
 'target_redshift',
 'target_moving',
 'target_keywords',
 'targetPosition_equinox',
 'targetPosition_coordinates_cval1',
 'targetPosition_coordinates_cval2',
 'telescope_geoLocationX',
 'telescope_geoLocationY',
 'telescope_geoLocationZ',
 'telescope_keywords',
 'instrument_keywords',
 'environment_seeing',
 'environment_humidity',
 'environment_elevation',
 'environment_tau',
 'environment_wavelengthTau',
 'environment_ambientTemp',
 'environment_photometric',
 'members',
 'typeCode',
 'metaChecksum',
 'obsID',
 'accMetaChecksum',
 'collection',
 'observationID',
 'algorithm_name',
 'type',
 'intent',
 'metaRelease',
 'proposal_id',
 'proposal_pi',
 'proposal_project',
 'proposal_title',
 'target_name',
 'target_type',
 'targetPosition_coordsys',
 'telescope_name',
 'requirements_flag',
 'instrument_name',
 'lastModified',
 'maxLastModified',
 'caomPlaneURI',
 'caomPublisherID',
 'calibrationLevel',
 

In [16]:
# Filter for both CFHT and HST data
prelim_filter = lambda table: (table['calibrationLevel'] == 3) & \
                              (table['dataProductType'] == 'image') & \
                              (table['quality_flag'] != 'junk')
        
cfht_filter = prelim_filter(cfht_results) & (cfht_results['target_name'] == 'D2') & \
    ([energy_bandpass_name.startswith('i.') or energy_bandpass_name.startswith('r.')
      for energy_bandpass_name in cfht_results['energy_bandpassName']])
    
hst_filter = prelim_filter(hst_results) & \
    ([target_name.startswith('COSMOS') for target_name in hst_results['target_name']])

filtered_cfht_results = cfht_results[cfht_filter]
filtered_hst_results = hst_results[hst_filter]

print('Number of CFHT results after filtering: {}'.format(len(filtered_cfht_results)))
print('Number of HST results after filtering: {}'.format(len(filtered_hst_results)))
print('Total number of results: {}'.format(len(filtered_cfht_results) + len(filtered_hst_results)))

Number of CFHT results after filtering: 69
Number of HST results after filtering: 29
Total number of results: 98


Now we can take a look at the results from the query. The results are returned as an astropy table, so we can use the `show_in_notebook` method to nicely render our results.

In [17]:
columns_subset = ['productID', 'collection', 'observationID', 'energy_bandpassName', 'calibrationLevel', \
                  'position_dimension_naxis1', 'position_dimension_naxis2', 'intent', 'target_name', \
                  'telescope_name', 'instrument_name', 'position_bounds', 'dataProductType']

filtered_cfht_results.sort('productID')
filtered_cfht_results[columns_subset].show_in_notebook()

idx,productID,collection,observationID,energy_bandpassName,calibrationLevel,position_dimension_naxis1,position_dimension_naxis2,intent,target_name,telescope_name,instrument_name,position_bounds,dataProductType
,,,,,,,,,,,,deg,
0,D2.I,CFHTMEGAPIPE,D2,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
1,D2.IQ.I,CFHTMEGAPIPE,D2.IQ,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
2,D2.IQ.R,CFHTMEGAPIPE,D2.IQ,r.MP9601,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
3,D2.R,CFHTMEGAPIPE,D2,r.MP9601,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
4,G025.150.118+02.207.I3,CFHTMEGAPIPE,G025.150.118+02.207,i.MP9703,3,24722,20483,science,D2,CFHT 3.6m,MegaPrime,[149.48339110622874 1.6817153538897978 149.48293764564525 2.737574824181491 150.75877389715515 2.737574824181462 150.75832043657107 1.6817153538897691],image
5,G025.150.118+02.207.R3,CFHTMEGAPIPE,G025.150.118+02.207,r.MP9602,3,24722,20483,science,D2,CFHT 3.6m,MegaPrime,[149.48339110622874 1.6817153538897978 149.48293764564525 2.737574824181491 150.75877389715515 2.737574824181462 150.75832043657107 1.6817153538897691],image
6,MegaPipe.300.184.I.MP9703,CFHTMEGAPIPE,MegaPipe.300.184,i.MP9703,3,10000,10000,science,D2,CFHT 3.6m,MegaPrime,[149.83327915952307 1.7419981411418342 149.833197957032 2.258012878859131 150.34961368209426 2.258012888021311 150.34953249584197 1.7419981482087643],image
7,MegaPipe.300.184.R.MP9602,CFHTMEGAPIPE,MegaPipe.300.184,r.MP9602,3,10000,10000,science,D2,CFHT 3.6m,MegaPrime,[149.83327915952307 1.7419981411418342 149.833197957032 2.258012878859131 150.34961368209426 2.258012888021311 150.34953249584197 1.7419981482087643],image
8,QD2.04AQ02.I,CFHTMEGAPIPE,QD2.04AQ02,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image


In [ ]:
# TODO: Remove this?
filtered_hst_results.sort('productID')
filtered_hst_results[columns_subset].show_in_notebook()


### Method 2: ADQL
The second method is to bundle the query and filtering in one using ADQL. In this case, we bundle both the CFHT and HST results into one query. In this case, we filter all results that contain the area of the coordinates within a certain radius.

In [19]:
query_vars = { 'ra': c.ra.degree, 'dec': c.dec.degree, 'radius': radius }
query = '''SELECT *
FROM caom2.Plane AS Plane 
JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID 
WHERE ( Observation.collection IN ('CFHTMEGAPIPE','HST')
AND ( Observation.target_name LIKE 'D2%' OR Observation.target_name LIKE 'COSMOS%' )
AND ( Plane.calibrationLevel = 3 )
AND ( Plane.dataProductType = 'image')
AND ( Plane.energy_bandpassName LIKE 'i.%' OR Plane.energy_bandpassName LIKE 'r.%' OR Plane.energy_bandpassName LIKE 'F%')
AND ( INTERSECTS( CIRCLE('ICRS',
{ra},
{dec},
{radius}),
Plane.position_bounds ) = 1 )
AND ( Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk' ))'''.format(**query_vars)

job = cadc.run_query(query, 'sync')
results = job.get_results()

results.sort('productID')

print('Total number of results: {}'.format(len(results)))
results[columns_subset].show_in_notebook()

Total number of results: 98


idx,productID,collection,observationID,energy_bandpassName,calibrationLevel,position_dimension_naxis1,position_dimension_naxis2,intent,target_name,telescope_name,instrument_name,position_bounds,dataProductType
,,,,,,,,,,,,deg,
0,D2.I,CFHTMEGAPIPE,D2,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
1,D2.IQ.I,CFHTMEGAPIPE,D2.IQ,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
2,D2.IQ.R,CFHTMEGAPIPE,D2.IQ,r.MP9601,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
3,D2.R,CFHTMEGAPIPE,D2,r.MP9601,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image
4,G025.150.118+02.207.I3,CFHTMEGAPIPE,G025.150.118+02.207,i.MP9703,3,24722,20483,science,D2,CFHT 3.6m,MegaPrime,[149.48339110622874 1.6817153538897978 149.48293764564525 2.737574824181491 150.75877389715515 2.737574824181462 150.75832043657107 1.6817153538897691],image
5,G025.150.118+02.207.R3,CFHTMEGAPIPE,G025.150.118+02.207,r.MP9602,3,24722,20483,science,D2,CFHT 3.6m,MegaPrime,[149.48339110622874 1.6817153538897978 149.48293764564525 2.737574824181491 150.75877389715515 2.737574824181462 150.75832043657107 1.6817153538897691],image
6,MegaPipe.300.184.I.MP9703,CFHTMEGAPIPE,MegaPipe.300.184,i.MP9703,3,10000,10000,science,D2,CFHT 3.6m,MegaPrime,[149.83327915952307 1.7419981411418342 149.833197957032 2.258012878859131 150.34961368209426 2.258012888021311 150.34953249584197 1.7419981482087643],image
7,MegaPipe.300.184.R.MP9602,CFHTMEGAPIPE,MegaPipe.300.184,r.MP9602,3,10000,10000,science,D2,CFHT 3.6m,MegaPrime,[149.83327915952307 1.7419981411418342 149.833197957032 2.258012878859131 150.34961368209426 2.258012888021311 150.34953249584197 1.7419981482087643],image
8,QD2.04AQ02.I,CFHTMEGAPIPE,QD2.04AQ02,i.MP9701,3,20341,21481,science,D2,CFHT 3.6m,MegaPrime,[149.59070574047269 1.6543167546229556 149.59031308263278 2.763731239911025 150.64207251736718 2.763731239911025 150.64167985952727 1.6543167546229842],image


Notice how we have the same number of results using both the two methods? Pretty  neat :)

## Selecting Data
Now let's choose two images, find a suitable cutout, download them, and display them. We will use the `get_data_urls` function to actually access the data.

In [21]:
# Selcting the first CFHT and HST images
cfht_result = results[(results['collection'] == 'CFHTMEGAPIPE')][0]
hst_result = results[(results['collection'] == 'HST')][0]

# Get links to the data
cfht_url = cadc.get_data_urls(results[(results['productID'] == cfht_result['productID'])])[0]
hst_url = cadc.get_data_urls(results[(results['productID'] == hst_result['productID'])])[0]

print('{} url: {}'.format(cfht_result['productID'], cfht_url))
print('{} url: {}'.format(hst_result['productID'], hst_url))

D2.I url: https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/D2.I.fits?RUNID=tduehvsqxkpsmdez
ibhm30030-PRODUCT url: https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/MAST/HST/product/ibhm30030_drz.fits?RUNID=ivleb4uvshvchlx2


The stucture of the url's can be broken into a few parts. Take 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHTSG/D2.I.fits?RUNID=tduehvsqxkpsmdez' as an example...
- `https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/`: This is the service url. The 'data/pub' ending accesses all public files. For files the requires authorization, there will be a redirect to 'data/auth' which require authorization
- `CFHTSG`: This is the archive name.
- `D2.I.fits`: This is the file name.
- `RUNID=f842u7t5dzws8mgy`: These are additional query parameters. I don't exactly know what this means!

The urls give access to the actual data, so now we can fetch the file, get data to calculate a cutout, open it with the astropy fits module, and display an image!


### Calculating cutouts

One thing to note is that the data products probably contain much more information than we actually need. In order to reduce the amount of data and total processing time, we can request cutouts of the fits files that only contain a slice of information. In order to do a cutout, we need the range of pixels we want, and/or the fits extensions that we want. In this particular case, we want to compare two different files, so we want to do a cutout based on the coordinates we want to look at. In order to do this, we recruit the `cadccutout` package which can transform coordinates to the pixel range used to perform the cutout.


In [78]:
from cadccutout import Transform
from astropy.io import fits

#TODO figure out why ext causes hst request to error
def cutout_pixels_from_position_with_ext(cutout_position, url, ext=0):
    transform = Transform()
    to_cutout_string = lambda pix_range: '[{}][{}:{},{}:{}]'.format(ext, pix_range[0][0], pix_range[0][1], 
                                                                pix_range[1][0], pix_range[1][1])
    header = fits.getheader(url, ext)
    hdu_pixels = transform.world_to_pixels([cutout_position], header)
    return to_cutout_string(hdu_pixels.get_ranges())

def cutout_pixels_from_position(cutout_position, url, ext=0):
    transform = Transform()
    to_cutout_string = lambda pix_range: '[{}:{},{}:{}]'.format(pix_range[0][0], pix_range[0][1], 
                                                                pix_range[1][0], pix_range[1][1])
    header = fits.getheader(url, ext)
    hdu_pixels = transform.world_to_pixels([cutout_position], header)
    return to_cutout_string(hdu_pixels.get_ranges())
    
# TODO: Cycle through ext in function instead?
coordinates = { 'ra': c.ra.degree, 'dec': c.dec.degree, 'radius': 0.001}
cutout_region_string = 'CIRCLE {ra} {dec} {radius}'.format(**coordinates)

cfht_pixels = cutout_pixels_from_position(cutout_region_string, cfht_url, ext=0)
hst_pixels = cutout_pixels_from_position(cutout_region_string, hst_url, ext=1)

print(cfht_pixels)
print(hst_pixels)

[10141:10181,10705:10745]
[1353:1098,503:560]


### Fetching the Data
With the cutout values, we can now download the data. There are many different methods to perform a cutout, but today we will add the cutout string to the parameter in the request. More information about requesting CADC data can be found here: http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/doc/data/.


In [79]:
import re
import requests
from astropy.wcs import WCS

#TODO: Make this read file if already there
#TODO: Catch 400 errors
def download_url_to_file(url, cutout=None, data_dir='data'):
    '''Takes a url, downloads the resource, saves it in the data directory with the
    name specified in the response Content Disposition, and returns the file path and name'''

    def get_filename_from_response(response):
        content_disposition = response.headers['Content-Disposition']
        return re.findall("filename=(.+)", content_disposition)[0]

    resp = requests.get(url, params={'cutout': cutout}) if cutout else requests.get(url)
    fname = get_filename_from_response(resp)
    file_name_and_path = '{data_dir}/{fname}'.format(data_dir=data_dir, fname=fname)
    # Note: wb will overwrite if something is already there
    with open(file_name_and_path, 'wb') as f:
        f.write(resp.content)
        
    return file_name_and_path

def get_image_data(file, ext=0):
    '''Returns the image data and wcs coordinate data of a fits file at the given extension'''
    with fits.open(file, ignore_missing_end=True) as hdulist:
        hdulist.info()
        w = WCS(hdulist[ext].header)
        image_data = hdulist[ext].data
    return image_data, w

def get_image_data_from_url(url, cutout=None, ext=0):
    '''Returns the image data and wcs coordinate data of a fits file at the given extension'''
    data_url = url + '&cutout={}'.format(cutout) if cutout else url
    with fits.open(data_url, ignore_missing_end=True) as hdulist:
        hdulist.info()
        w = WCS(hdulist[ext].header)
        image_data = hdulist[ext].data
    return image_data, w



# To save to a file then read to image data
hst_fname = download_url_to_file(hst_url, cutout=hst_pixels)
cfht_fname = download_url_to_file(cfht_url, cutout=cfht_pixels)

hst_image_data, hst_w = get_image_data('{fname}'.format(fname=hst_fname), ext=1)
cfht_image_data, cfht_w = get_image_data('{fname}'.format(fname=cfht_fname), ext=0)


In [70]:
# To read from url
# Use this for myBinder
hst_image_data, hst_w = get_image_data_from_url(hst_url, cutout=hst_pixels, ext=1)
cfht_image_data, cfht_w = get_image_data_from_url(cfht_url, cutout=cfht_pixels, ext=0)


Filename: /home/badune/.astropy/cache/download/py3/e1b39bf7fdb40f6ab0ca8950f320bf6e
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     809   ()      
  1  SCI           1 ImageHDU        72   (1098, 968)   float32   


TypeError: buffer is too small for requested array

The astropy WCS module gives position information about the image pixels. We can take a look at the position range of the HST and CFHT images and check to see how close the cutout of the CFHT image is to the HST image.

In [65]:
def print_position_range(image_data, w):
    px = [0, image_data.shape[0]-1]
    py = [0, image_data.shape[1]-1]
    ra, dec = w.all_pix2world(px, py, 0)
    print('ra: ', ra)
    print('dec: ', dec)
    
print('HST Position Range')
print_position_range(hst_image_data, hst_w)

print('CFHT Position Range')
print_position_range(cfht_image_data, cfht_w)

HST Position Range


NameError: name 'hst_image_data' is not defined

Looks pretty close! Although seems like one axis is flipped... but now we can print out the images!!! First lets print out both side by side.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#TODO: Add lables and titles!
#TODO: look and see how to make imshow better
def print_image_stats(image_data, title):
    print(title)
    stats = {'min': np.nanmin(image_data), 'max': np.nanmax(image_data), 'mean': np.nanmean(image_data), 
            'std': np.nanstd(image_data)}
    print('Min: {min} \nMax: {max} \nMean: {mean}\nStdev: {std}'.format(**stats))
    
def plot_image(image_data, wcs):
    ax = plt.subplot(projection=wcs)
#     ax.set_xlim(-0.5, image_data.shape[1] - 0.5)
#     ax.set_ylim(-0.5, image_data.shape[0] - 0.5)
    ax.imshow(image_data, vmin=0, vmax=0.1, origin='lower', cmap='gray')
    ax.grid(color='white', ls='solid')
    plt.xlabel('Galactic Longitude')
    plt.ylabel('Galactic Latitude')
    
def plot_images(image_data_a, wcs_a, image_data_b, wcs_b):
    #TODO: Fix axes
    fig = plt.figure(figsize=(20,10))
    ax1 = plt.subplot(121, projection=wcs_a)
    ax1.imshow(image_data_a, vmin=0, vmax=5, origin='lower', cmap='gray')
    ax2 = plt.subplot(122, projection=wcs_b)
    ax2.imshow(image_data_b, vmin=0, vmax=0.05, origin='lower', cmap='gray')
    ax1.grid(color='white', ls='solid')
    ax2.grid(color='white', ls='solid')
    plt.xlabel('Galactic Longitude')
    plt.ylabel('Galactic Latitude')
    plt.show()
    
def plot_dual_wcs_images(image_data_a, wcs_a, image_data_b, wcs_b):
    fig = plt.figure(figsize=(20,10))
    ax = plt.subplot(projection=wcs_a)
    ax.set_xlim(-0.5, image_data_a.shape[1] - 0.5)
    ax.set_ylim(-0.5, image_data_a.shape[0] - 0.5)
    ax.imshow(image_data_a, vmin=0, vmax=5, origin='lower')
    ax.contour(image_data_b, transform=ax.get_transform(wcs_b), levels=np.logspace(-4, -2, num=2,  base=2,), colors='red')
    ax.grid(color='white', ls='solid')
    plt.xlabel('Galactic Longitude')
    plt.ylabel('Galactic Latitude')


In [ ]:
plot_images(cfht_image_data, cfht_w, hst_image_data, hst_w)

In [ ]:
title = 'HST Image Stats'
print_image_stats(hst_image_data, title)

title = 'CFHT Image Stats'
print_image_stats(cfht_image_data, title)

In [ ]:
plot_dual_wcs_images(cfht_image_data, cfht_w, hst_image_data, hst_w)